In [2]:
pip install streamlit

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [80]:
%%writefile C:\weatherPyAI\WeaterApp.py
import streamlit as st
from PIL import Image

# ---------------- 사이드바 화면 구성 ------------------------
# 지역(날씨) 선택 => 날씨 호출
# 온도 선택 => 화씨, 섭씨 호출
# 지역(시간) 선택 =>  시간 호출 
st.sidebar.title('사이드바')
st.sidebar.header('오늘의 날씨🌅')
area_weather = st.sidebar.text_input('Area', value='차차네', max_chars=15)
area_temp = st.sidebar.text_input('온도', value='뭉치도', max_chars=2)

st.sidebar.header("셀렉트박스 사용")
selectbox_options = ['화씨', '섭씨', '-'] # 온도 선택
your_option = st.sidebar.selectbox('온도 선택', selectbox_options, index=2) # 선택 박스
st.sidebar.write('**당신의 선택**:', your_option)


# --------------------------- 메인(Main) 화면 구성 --------------------

# 아래에 test.py에 있는 코드 추가
import streamlit as st
import openai
import os
import requests
import json
import re
import csv

# Chat Completion API를 이용해 사용자 입력에 따라 함수를 호출하고 응답하는 함수
# API 키 설정
openai.api_key = os.environ["OPENAI_API_KEY"]
WEATHER_API_KEY = os.environ["WEATHER_API_KEY"]

def get_current_weather(location, unit="섭씨"):
    
    reg = re.compile(r'[a-zA-Z]') # 영어 입력인지를 검사하는 정규식
    
    if reg.match(location):  # 만약 입력한 위치에 영어 문자가 포함되어 있다면
        city = location  # 해당 위치를 그대로 사용합니다.
    else:  # 입력한 위치에 영어 문자가 포함되어 있지 않다면
        city_names_korean_to_english = {}  # 한국어 도시 이름과 영어 번역을 저장할 딕셔너리를 생성합니다.

        # CSV 파일에서 도시 이름을 읽어와 딕셔너리를 채웁니다.
        with open('./korean_city.csv', 'r', encoding='utf-8') as file:
            reader = csv.reader(file)
            for row in reader:
                korean_city, english_city = row
                city_names_korean_to_english[korean_city] = english_city
    
        # 한국어 도시 이름을 영어로 번역합니다.
        city = city_names_korean_to_english.get(location)
        
    WEATHER_API_KEY = os.environ["WEATHER_API_KEY"] # API 키 지정
    
    url = "http://api.weatherapi.com/v1/current.json"
    parameters = {"key":WEATHER_API_KEY, "q":city}
    
    r = requests.get(url, params=parameters)
    current_weather = r.json()
    
    name = current_weather['location']['name'] # 설정 지역
    localtime = current_weather['location']['localtime'] # 날짜 및 시각
    temp_c = current_weather['current']['temp_c'] # 섭씨온도
    temp_f = current_weather['current']['temp_f'] # 화씨온도
    condition_text = current_weather['current']['condition']['text'] # 날씨 상태
    
    # unit 지정에 따라서 섭씨온도 혹은 화씨온도를 지정
    if unit == "섭씨":
        temp = temp_c
    elif unit == "화씨":
        temp = temp_f
    else:
        unit == "섭씨"
        temp = temp_c
        
        
    weather_info = {
        "location": name,
        "temperature": temp,
        "unit": unit,
        "current weather": condition_text,
        "local time": localtime
    }
    
    return json.dumps(weather_info, ensure_ascii=False) # JSON 형식으로 반환

# Chat Completion API를 이용해 사용자 입력에 따라 함수를 호출하고 응답하는 함수
def run_conversation(user_query):
    # 사용자 입력
    messages = [{"role":"user", "content":user_query}]
    
    # 함수 정보 입력
    functions = [
        {
            "name":"get_current_weather",
            "description": "도시 이름을 입력해 현재 날씨, 날짜, 시각, 몇 시인지 가져오기",
            "parameters": {
                "type": "object",
                "properties":{
                    "location": {
                        "type": "string",
                        "description": "도시 이름, 예를 들면, 서울, 부산, 대전",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["섭씨", "화씨"],
                        "description": "온도 단위로 섭씨 혹은 화씨",
                    },
                },
                "required": ["location"], # 필수 입력 변수 지정
            }
        }
    ]
    
    # 1단계: 사용자 입력과 함수 정보를 Chat Completions API 모델로 보내기
    response = openai.ChatCompletion.create( # Chat Completions API 모델로 보내기
        # model="gpt-3.5-turbo"
        model="gpt-3.5-turbo",
        messages=messages,
        functions=functions,
        function_call="auto"
    )
    
    # 2단계: 응답 생성
    response_message = response["choices"][0]["message"] # 모델의 응답 메시지
    
    if response_message.get("function_call"): # 응답이 함수 호출인지 확인하기
        # 3단계: JSON 객체를 분석해 함수 이름과 인수를 추출한 후에 함수 호출
        # (주의: JSON 응답이 항상 유효하지 않을 수 있음)
        
        # 호출할 함수 이름을 지정
        # (아래는 하나의 함수를 지정했지만 여러 함수 지정 가능)
        available_functions = {"get_current_weather": get_current_weather}
        
        # 함수 이름 추출
        function_name = response_message["function_call"]["name"]
        
        # 호출할 함수 선택
        function_to_call = available_functions[function_name]
        
        # 함수 호출을 위한 인수 추출
        function_args = json.loads(response_message["function_call"]["arguments"])
        
        # 함수 호출 및 반환 결과 받기
        function_response = function_to_call(
            location=function_args.get("location"), # 인수 지정
            unit=function_args.get("unit")
        )
        print("[호출한 함수의 응답 결과]\n", function_response)
        
        # 4단계: 함수 호출 결과를 기존 메시지에 추가하고,
        #        Chat Completions API 모델로 보내 응답받기
        
        # 함수 호출 결과를 기존 메시지에 추가히기
        messages.append(response_message) # 기존 messages에 조력자 응답 추가
        messages.append(                  # 함수와 함수 호출 결과 추가
            {
                "role": "function",       # roll: function으로 지정 
                "name": function_name,    # name: 호출할 함수 이름 지정
                "content": function_response, # content: 함수 호출 결과 지정
            }
        )
        # 함수 호출 결과를 추가한 메시지를 Chat Completions API 모델로 보내 응답받기
        second_response = openai.ChatCompletion.create(
            # model="get-3.5-turbo",
            model="gpt-3.5-turbo",
            messages=messages,
        )
        return second_response # 두 번째 응답 반환
    
    return response_message # 응답 메시지 반환


# Streamlit 애플리케이션 시작
st.title("대한민국 도시 날씨 정보 알리미🌅")
st.markdown("<br>", unsafe_allow_html=True)
st.markdown("<br>", unsafe_allow_html=True)
# 사용자 입력 받기
user_input = st.text_input("날씨에 대해 물어보세요.")
st.markdown("<br>", unsafe_allow_html=True)
# 사용자 입력에 따라 응답 생성
if user_input:
    response = run_conversation(user_input)
    response_content = response["choices"][0]["message"]["content"]
    st.write(response_content) # 응답 표시

Overwriting C:\weatherPyAI\WeaterApp.py


In [20]:
# import streamlit as st

# st.title("날씨를 알려주는 어플")
# #-----------------이미지 업로드 --------------------------#
# uploaded_file = st.file_uploader("이미지 파일을 선택해 주세요.", type='image')
# if uploade_file is not None:

In [59]:
import pandas as pd

korean_cities = {
    "서울": "Seoul", "부산": "Busan", "인천": "Incheon", "대구": "Daegu", "대전": "Daejeon",
    "광주": "Gwangju", "수원": "Suwon", "울산": "Ulsan", "창원": "Changwon", "고양": "Goyang",
    "용인": "Yongin", "성남": "Seongnam", "청주": "Cheongju", "안산": "Ansan", "안양": "Anyang",
    "전주": "Jeonju", "천안": "Cheonan", "남양주": "Namyangju", "화성": "Hwaseong", "파주": "Paju",
    "평택": "Pyeongtaek", "의정부": "Uijeongbu", "시흥": "Siheung", "김해": "Gimhae", "구미": "Gumi",
    "광명": "Gwangmyeong", "김포": "Gimpo", "제주": "Jeju", "양산": "Yangsan", "나주": "Naju",
    "아산": "Asan", "익산": "Iksan", "양주": "Yangju", "동두천": "Dongducheon", "이천": "Icheon",
    "구리": "Guri", "여수": "Yeosu", "충주": "Chungju", "안동": "Andong", "김천": "Gimcheon",
    "목포": "Mokpo", "경산": "Gyeongsan", "정읍": "Jeongeup", "거제": "Geoje", "진주": "Jinju",
    "포항": "Pohang", "완주": "Wanju", "단양": "Danyang", "부여": "Buyeo", "보령": "Boryeong",
    "태안": "Taean", "당진": "Dangjin", "서산": "Seosan", "홍성": "Hongseong", "청양": "Cheongyang",
    "영덕": "Yeongdeok", "울진": "Uljin", "포천": "Pocheon", "양평": "Yangpyeong", "하남": "Hanam",
    "의왕": "Uiwang", "오산": "Osan", "하동": "Hadong", "고성": "Goseong", "영암": "Yeongam",
    "신안": "Sinan", "청송": "Cheongsong", "영월": "Yeongwol", "고창": "Gochang", "무주": "Muju",
    "화순": "Hwasun", "진안": "Jinan", "영광": "Yeonggwang", "장성": "Jangseong", "함평": "Hampyeong",
    "강진": "Gangjin", "신안": "Sinan", "담양": "Damyang", "곡성": "Gokseong", "장흥": "Jangheung",
    "해남": "Haenam", "보성": "Boseong", "고흥": "Goheung", "강화": "Ganghwa", "옹진": "Ongjin",
    "진도": "Jindo", "함안": "Haman", "고창": "Gochang", "영광": "Yeonggwang", "장성": "Jangseong",
    "함평": "Hampyeong", "강진": "Gangjin", "신안": "Sinan", "담양": "Damyang", "곡성": "Gokseong",
    "장흥": "Jangheung", "해남": "Haenam", "보성": "Boseong", "고흥": "Goheung", "강화": "Ganghwa",
    "옹진": "Ongjin", "진도": "Jindo", "함안": "Haman", "고령": "Goryeong", "성주": "Seongju",
    "의성": "Uiseong", "청송": "Cheongsong", "영양": "Yeongyang", "영덕": "Yeongdeok", "울릉": "Ulleung",
    "독도": "Dokdo", "울진": "Uljin", "상주": "Sangju", "문경": "Mungyeong", "안동": "Andong",
    "영주": "Yeongju", "예천": "Yecheon", "경주": "Gyeongju", "청도": "Cheongdo", "군위": "Gunwi",
    "의령": "Uiryeong", "청송": "Cheongsong", "영덕": "Yeongdeok", "영양": "Yeongyang", "영주": "Yeongju",
    "영천": "Yeongcheon", "예천": "Yecheon", "울릉": "Ulleung", "울진": "Uljin", "인제": "Inje",
    "정선": "Jeongseon", "철원": "Cheorwon", "태백": "Taebaek", "평창": "Pyeongchang", "홍천": "Hongcheon",
    "화천": "Hwacheon", "횡성": "Hoengseong", "강릉": "Gangneung", "동해": "Donghae", "삼척": "Samcheok",
    "속초": "Sokcho", "양구": "Yanggu", "양양": "Yangyang", "인제": "Inje", "정선": "Jeongseon",
    "철원": "Cheorwon", "태백": "Taebaek", "세종": "Sejong", "원주": "Wonju", "춘천": "Chuncheon", "천안": "Cheonan",
    "수원": "Suwon","안동": "Andong", "대구": "Daegu", "제주": "Jeju", "포항": "Pohang", "광주": "Gwangju",
    "창원": "Changwon", "평택": "Pyeongtaek", "청주": "Cheongju", "안산": "Ansan", "양산": "Yangsan",
    "서울": "Seoul", "부산": "Busan", "대전": "Daejeon", "울산": "Ulsan", "인천": "Incheon",
    "세종": "Sejong", "강릉": "Gangneung", "원주": "Wonju", "정선": "Jeongseon", "동해": "Donghae",
    "영월": "Yeongwol", "삼척": "Samcheok", "태백": "Taebaek", "속초": "Sokcho", "홍천": "Hongcheon",
    "횡성": "Hoengseong", "철원": "Cheorwon", "화천": "Hwacheon", "양구": "Yanggu", "인제": "Inje",
    "고성": "Goseong", "양양": "Yangyang", "춘천": "Chuncheon", "원주": "Wonju", "강릉": "Gangneung",
    "동해": "Donghae", "삼척": "Samcheok", "속초": "Sokcho", "홍천": "Hongcheon", "횡성": "Hoengseong",
    "영월": "Yeongwol", "정선": "Jeongseon", "철원": "Cheorwon", "화천": "Hwacheon", "양구": "Yanggu",
    "인제": "Inje", "고성": "Goseong", "양양": "Yangyang", "청양": "Cheongyang", "단양": "Danyang",
    "보은": "Boeun", "영동": "Yeongdong", "영춘": "Yeongchun", "영동": "Yeongdong", "보은": "Boeun",
    "옥천": "Okcheon", "음성": "Eumseong", "제천": "Jecheon", "증평": "Jeungpyeong", "진천": "Jincheon",
    "괴산": "Goesan", "단양": "Danyang", "보은": "Boeun", "영동": "Yeongdong", "옥천": "Okcheon",
    "음성": "Eumseong", "제천": "Jecheon", "증평": "Jeungpyeong", "진천": "Jincheon", "괴산": "Goesan",
    "대천": "Daecheon", "보령": "Boryeong", "서천": "Seocheon", "부여": "Buyeo", "금산": "Geumsan",
    "논산": "Nonsan", "당진": "Dangjin", "태안": "Taean", "예산": "Yesan", "홍성": "Hongseong",
    "청양": "Cheongyang", "공주": "Gongju", "금산": "Geumsan", "논산": "Nonsan", "당진": "Dangjin",
    "태안": "Taean", "예산": "Yesan", "홍성": "Hongseong", "청양": "Cheongyang", "공주": "Gongju",
    "부여": "Buyeo", "금산": "Geumsan", "논산": "Nonsan", "당진": "Dangjin", "태안": "Taean",
    "예산": "Yesan", "홍성": "Hongseong", "청양": "Cheongyang", "공주": "Gongju", "보령": "Boryeong",
    "대천": "Daecheon", "서천": "Seocheon", "부여": "Buyeo", "금산": "Geumsan", "논산": "Nonsan",
    "당진": "Dangjin", "태안": "Taean", "예산": "Yesan", "홍성": "Hongseong", "청양": "Cheongyang",
    "공주": "Gongju", "보령": "Boryeong", "대천": "Daecheon", "서천": "Seocheon", "연천": "Yeoncheon",
    "포천": "Pocheon", "가평": "Gapyeong"
}

# 딕셔너리를 데이터프레임으로 변환
df = pd.DataFrame(list(korean_cities.items()), columns=['한글 도시 이름', '영어 도시 이름'])

# 데이터프레임을 CSV 파일로 저장
df.to_csv('korean_citi.csv', index=False)

In [27]:
pip install streamlit openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
